# ASSIGNMENT 12

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import sqlite3
from pandasql import sqldf

In [2]:
# Given Url
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
# Reading the data from the given url
df = pd.read_csv(url)
# Naming the columns of the dataframe
df.columns=['Age','WorkClass','FnlWgt','Education','EducationNum','Marital-Status','Occupation','Relationship','Race','Sex','Capital-gain','Capital-loss','Hours-Per-Week','Native-Country','Income']
# Printing the rows of the dataframe
df.head(2)

,Age,WorkClass,FnlWgt,Education,EducationNum,Marital-Status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-Per-Week,Native-Country,Income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [3]:
# Defining a function to eliminate the blank spaces of the columns 
def trimAllColumns(df):
    trimStrings = lambda x: x.strip() if type(x) is str else x
    return df.applymap(trimStrings)
#Trimming out the blank spaces 
df = trimAllColumns(df)
df.head(2)


,Age,WorkClass,FnlWgt,Education,EducationNum,Marital-Status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-Per-Week,Native-Country,Income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [4]:
#Create a sql db from adult dataset and name it sqladb

# Connecting to a database
con = sqlite3.connect('sqladb.db')
cur = con.cursor()
# Copying the contents of the dataframe to the sqladb table
df.to_sql('sqladb', con, if_exists='replace', index=False)

#Q.1 Select 10 records from the adult sqladb
pd.read_sql('select * from sqladb LIMIT 10', con)

,Age,WorkClass,FnlWgt,Education,EducationNum,Marital-Status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-Per-Week,Native-Country,Income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K


In [5]:
# Q2. Show me the average hours per week of all men who are working in private sector
pd.read_sql_query("SELECT round(avg([Hours-Per-Week]),3) as AvgHoursPerWeek FROM sqladb WHERE Sex='Male' AND WorkClass='Private'",con )

,AvgHoursPerWeek
0,42.221


In [6]:
# Q3. Show me the frequency table for education, occupation and relationship, separately
q1 = """SELECT Education, COUNT(Education)AS Frequency

  FROM sqladb

  GROUP BY Education

  ORDER BY  COUNT(Education) DESC"""

q2 = """SELECT Occupation, COUNT(Occupation)AS Frequency

  FROM sqladb

  GROUP BY Occupation

  ORDER BY  COUNT(Occupation) DESC"""

q3 = """SELECT Relationship, COUNT(Relationship)AS Frequency

  FROM sqladb

  GROUP BY Relationship

  ORDER BY  COUNT(Relationship) DESC"""


print("FREQUENCY TABLE FOR EDUCATION:\n","-"*60)
print(pd.read_sql_query(q1,con))
print("\nFREQUENCY TABLE FOR OCCUPATION:\n","-"*60)
print(pd.read_sql_query(q2,con))
print("\nFREQUENCY TABLE FOR RELATIONSHIP:\n","-"*60)
print(pd.read_sql_query(q3,con))

FREQUENCY TABLE FOR EDUCATION:
 ------------------------------------------------------------
       Education  Frequency
0        HS-grad      10501
1   Some-college       7291
2      Bachelors       5354
3        Masters       1723
4      Assoc-voc       1382
5           11th       1175
6     Assoc-acdm       1067
7           10th        933
8        7th-8th        646
9    Prof-school        576
10           9th        514
11          12th        433
12     Doctorate        413
13       5th-6th        333
14       1st-4th        168
15     Preschool         51

FREQUENCY TABLE FOR OCCUPATION:
 ------------------------------------------------------------
           Occupation  Frequency
0      Prof-specialty       4140
1        Craft-repair       4099
2     Exec-managerial       4066
3        Adm-clerical       3769
4               Sales       3650
5       Other-service       3295
6   Machine-op-inspct       2002
7                   ?       1843
8    Transport-moving       1597
9   Ha

In [7]:
# Q4. Are there any people who are married, working in private sector and having a masters degree
# Finding out the various categories of Marital-Status
q4 = """SELECT [Marital-Status], COUNT([Marital-Status])AS Frequency

  FROM sqladb

  GROUP BY [Marital-Status]

  ORDER BY  COUNT([Marital-Status]) DESC"""

pd.read_sql_query(q4,con)

,Marital-Status,Frequency
0,Married-civ-spouse,14976
1,Never-married,10682
2,Divorced,4443
3,Separated,1025
4,Widowed,993
5,Married-spouse-absent,418
6,Married-AF-spouse,23


In [16]:
# Q4. Are there any people who are married, working in private sector and having a masters degree

# Structuring the query as per the given conditions
q5 = """SELECT Education,[Marital-Status],WorkClass FROM sqladb 

        WHERE [Marital-Status] IN  ('Married-civ-spouse','Married-spouse-absent','Married-AF-spouse') 
        
        AND ([WorkClass] = 'Private' AND [Education] = 'Masters')"""

# Printing such records
print(pd.read_sql_query(q5,con))

# Query Execution
cur.execute(q5)

# Code block to find if the records fulfilling the given conditions exist
if cur.fetchone():
    print("Record with the given conditions found. Hence such a person exists")

else:
    print("Record with the given conditions not found.Hence such a person does not exist")

    Education         Marital-Status WorkClass
0     Masters     Married-civ-spouse   Private
1     Masters     Married-civ-spouse   Private
2     Masters     Married-civ-spouse   Private
3     Masters     Married-civ-spouse   Private
4     Masters     Married-civ-spouse   Private
5     Masters     Married-civ-spouse   Private
6     Masters     Married-civ-spouse   Private
7     Masters     Married-civ-spouse   Private
8     Masters     Married-civ-spouse   Private
9     Masters     Married-civ-spouse   Private
10    Masters     Married-civ-spouse   Private
11    Masters     Married-civ-spouse   Private
12    Masters     Married-civ-spouse   Private
13    Masters     Married-civ-spouse   Private
14    Masters     Married-civ-spouse   Private
15    Masters     Married-civ-spouse   Private
16    Masters     Married-civ-spouse   Private
17    Masters     Married-civ-spouse   Private
18    Masters     Married-civ-spouse   Private
19    Masters     Married-civ-spouse   Private
20    Masters

In [9]:
# Q5. What is the average, minimum and maximum age group for people working in different sectors
q6 = """SELECT AVG(Age) AS Average_Age, 
                MIN(Age) AS Min_Age, 
                MAX(Age) AS Max_Age,
                WorkClass AS WorkClass
               
          FROM sqladb

          GROUP BY [WorkClass] """

pd.read_sql_query(q6,con)

,Average_Age,Min_Age,Max_Age,WorkClass
0,40.960240,17,90,?
1,42.590625,17,90,Federal-gov
2,41.751075,17,90,Local-gov
3,20.571429,17,30,Never-worked
4,36.797585,17,90,Private
5,46.017025,17,84,Self-emp-inc
6,44.969697,17,90,Self-emp-not-inc
7,39.436392,17,81,State-gov
8,47.785714,19,72,Without-pay


In [10]:
# Q6. Calculate age distribution by country

q7 = """SELECT [Native-Country] as Country,
                AVG(Age) AS Average_Age, 
                MIN(Age) AS Min_Age, 
                MAX(Age) AS Max_Age,
                COUNT(Age) as Count,
                AVG(Age*Age) - AVG(Age)*AVG(Age) AS variance
                
                               
          FROM sqladb

          GROUP BY [Native-Country] """

pd.read_sql_query(q7,con)

,Country,Average_Age,Min_Age,Max_Age,Count,variance
0,?,38.725557,17,90,583,153.615933
1,Cambodia,37.789474,18,65,19,116.166205
2,Canada,42.545455,17,80,121,225.586777
3,China,42.533333,22,75,75,154.808889
4,Columbia,39.711864,18,75,59,181.628842
5,Cuba,45.768421,21,82,95,179.988476
6,Dominican-Republic,37.728571,18,78,70,169.083469
7,Ecuador,36.642857,21,90,28,208.158163
8,El-Salvador,34.132075,17,79,106,147.095764
9,England,41.155556,17,90,90,201.620247


In [11]:
# Q7. Compute a new column as 'Net-Capital-Gain' from the two columns 'capital-gain' and 'capital-loss'

# Computing a new column Net-Capital-Gain
q8 = """SELECT *,([Capital-gain]-[Capital-loss]) as [Net-Capital-Gain] from sqladb"""
pd.read_sql_query(q8,con)



,Age,WorkClass,FnlWgt,Education,EducationNum,Marital-Status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-Per-Week,Native-Country,Income,Net-Capital-Gain
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,0
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K,0
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K,0
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K,14084
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K,5178
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K,0


In [12]:
# Adding a new column to the database
cur.execute("alter table sqladb add column [Net-Capital-Gain] 'int'")
cur.execute(" UPDATE sqladb  SET [Net-Capital-Gain]=[Capital-Gain]-[Capital-Loss]") 

In [13]:
pd.read_sql('select * from sqladb LIMIT 10', con)

,Age,WorkClass,FnlWgt,Education,EducationNum,Marital-Status,Occupation,Relationship,Race,Sex,Capital-gain,Capital-loss,Hours-Per-Week,Native-Country,Income,Net-Capital-Gain
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K,0
5,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K,0
6,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K,0
7,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K,14084
8,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K,5178
9,37,Private,280464,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,80,United-States,>50K,0
